In [2]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train_df = pd.read_csv("D:/PythonSamples/Mercari/train.tsv", sep='\t', header=0,engine="python")

In [3]:
test_df = pd.read_csv("D:/PythonSamples/Mercari/test.tsv", sep='\t', header=0,engine="python")

In [4]:
train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home DÃ©cor/Home DÃ©cor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [5]:
test_df.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [6]:
train_df['category_name'].fillna('NoValue', inplace=True)

In [7]:
train_df['brand_name'].fillna('NoValue', inplace=True)

In [8]:
train_df['item_description'].fillna('NoValue', inplace=True)

In [9]:
train_df = train_df[train_df['price'] != 0.0]

In [10]:
test_df['price'] = 999999.00 

In [11]:
test_df = test_df[['test_id','name','item_condition_id','category_name','brand_name','shipping','item_description','price']]

In [12]:
test_df['category_name'].fillna('NoValue', inplace=True)

In [13]:
test_df['brand_name'].fillna('NoValue', inplace=True)

In [14]:
x = train_df[['category_name','name']]
y = test_df[['category_name','name']]

In [15]:
category_df = x
category_df = category_df.append(y)

In [16]:
category_df['name'] = category_df['name'].astype(str).str.split(' ').apply(list)

In [17]:
category_ref = category_df.groupby('category_name')['name'].apply(list)

In [18]:
category_ref_df = pd.DataFrame({'category_name':category_ref.index, 'name':category_ref.values}) 

In [19]:
def flattenlistname(row):
    listoflist = row['name']
    #finallist = reduce(lambda x,y: x+y,listoflist)
    finallist = ' '.join(str(r) for v in listoflist for r in v)
    #print(row['category_name'])
    return finallist

In [20]:
category_ref_df['name'] = category_ref_df.apply(flattenlistname, axis = 1)

In [21]:
category_ref_df['name'] = category_ref_df['name'].str.lower()

In [22]:
def removeDupl(text):
    #print(text)
    list_ = re.sub('[^A-Za-z ]+', '',text)
    list_val = list_.split(' ')
    #print(list_val)
    finallist = list(set(list_val))
    final_list1 = []
    for i in finallist:
        #print(i)
        if len(i) > 1:
            final_list1.append(i)
    return final_list1

In [23]:
category_ref_df['name'] = category_ref_df['name'].apply(lambda x: removeDupl(x))

In [24]:
def cleanUnwantedDesc(row):
    item_desc = str(row)
    item_desc = re.sub('[^A-Za-z ]+', '',item_desc)
    tokens = nltk.word_tokenize(item_desc)
    tokens
    tagged = nltk.pos_tag(tokens)
    cleansed = ''
    for v,k in tagged:
        if k == 'NN' or k == 'NNP' or k == 'NNPS' or k == 'NNS':
            cleansed=cleansed+v+' '
    return cleansed.rstrip().split()

In [25]:
category_ref_df['name'] = category_ref_df['name'].apply(lambda x: cleanUnwantedDesc(x))

In [26]:
category_ref_df = category_ref_df[category_ref_df.category_name != 'NoValue']

In [27]:
category_ref_df.head()

,category_name,name
0,Beauty/Bath & Body/Bath,"[jet, bodyface, custom, misfit, rack, citrus, ..."
1,Beauty/Bath & Body/Bathing Accessories,"[soap, jet, hardware, dot, xl, custom, aerie, ..."
2,Beauty/Bath & Body/Cleansers,"[saras, glotherapeutics, sets, merlot, crystel..."
3,Beauty/Bath & Body/Other,"[soap, jet, treats, pregnancy, hd, day, boxes,..."
4,Beauty/Bath & Body/Scrubs & Body Treatments,"[soap, round, apples, sets, sakura, brittany, ..."


In [28]:
train_df_cat = train_df

In [29]:
#train_df_cat[train_df_cat.category_name == 'NoValue'].head()

In [31]:
import spacy
def findcategory(list_v):
    catval = list_v['category_name']
    if catval == 'NoValue':
        nlp = spacy.load('en')
        wval = list_v['name']
        print(list_v['train_id'])
        wval = wval.lower()
        fnl = {}
        fnl_list = {}
        for index, row in category_ref_df.iterrows():
            c1 = nlp(wval)
            c2 = ' '.join(str(v) for v in row['name']) 
            c2 = nlp(c2)
            resu = c1.similarity(c2)
            #print(resu)
            fnl[row['category_name']] = resu
        for k,v in fnl.items():
            if v == max(fnl.values()):
                return str(k)
    else:
        return str(catval)

In [ ]:
import multiprocessing as mp
import pandas.util.testing as pdt

def process(df):
    df['category_name'] = df.apply(findcategory, axis=1)
    return df

p = mp.Pool(processes=10)
split_dfs = np.array_split(train_df_cat,10)
pool_results = p.map(process, split_dfs)
p.close()
p.join()

# merging parts processed by different processes
parts = pd.concat(pool_results, axis=0)

# merging newly calculated parts to big_df
train_df_cat = pd.concat([train_df_cat, parts], axis=1)

# checking if the dfs were merged correctly
pdt.assert_series_equal(parts['train_id'], big_df['train_id'])

In [ ]:
#train_df_cat['category_name'] = train_df_cat.apply(findcategory, axis = 1)

In [ ]:
import multiprocessing as mp
import numpy as np
import pandas as pd

def testfun(x):
    m = x['id']+2
    return m

if __name__ == "__main__":
    x = [0,1,2,3,4,5,6,7,8,9]
    y = ['a','b','c','d','e','f','g','h','i','j']
    var = pd.DataFrame({'$id':x,'$name':y})
    var.columns = ['id','name']
    
    def process(df):
        df['id'] = df.apply(testfun, axis = 1)
        return df
    
    p = mp.Pool(processes=5)
    split_dfs = np.array_split(var,5)
    pool_results = p.map(process, split_dfs)
    p.close()
    p.join()

pool_results

In [4]:
var

,id,name
0,0,a
1,1,b
2,2,c
3,3,d
4,4,e
5,5,f
6,6,g
7,7,h
8,8,i
9,9,j
